In [28]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import requests
from zipfile import ZipFile
from io import BytesIO

from config import FIRSTRATE_URL

In [29]:
# Download minute data for SPX and EOD for VIX
r = requests.get(FIRSTRATE_URL + "/9841", stream=True)
spx_file = ZipFile(BytesIO(r.raw.read()))
r = requests.get(FIRSTRATE_URL + "/14722", stream=True)
spx_updates = ZipFile(BytesIO(r.raw.read()))

r = requests.get("https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv", stream=True)
vix_file = BytesIO(r.raw.read())

In [30]:
def CalcRV(prices: pd.Series) -> pd.Series:
    """
    Calculates daily realized variation
    
    Paramters
    ---------
    prices : pd.Series
        Series of prices from a single trading day
    
    Returns
    -------
    Float
        Realized variation
    """
    log_returns = np.log(prices) - np.log(prices.shift(1))
    return np.sum(log_returns**2)

In [31]:
SPX = (pd.read_csv(spx_file.open("SPX_1min.txt"), names=["date", "open", "high", "low", "close", "volume"], parse_dates=["date"])
       .set_index("date"))
SPX_updates = (pd.read_csv(spx_updates.open("SPX_1-min.txt"), names=["date", "open", "high", "low", "close", "volume"], parse_dates=["date"])
       .set_index("date"))

# Realized volatility in annualized percentage terms
RV = (SPX.append(SPX_updates)
      .drop_duplicates()
      ["close"]
      .groupby(lambda x: x.date)
      .agg(CalcRV)
      .transform(lambda x: np.sqrt(x) * np.sqrt(252) * 100))

# EOD VIX values
vix = (pd.read_csv(vix_file, header=0, names=["date", "open", "high", "low", "close"], parse_dates=["date"])
       .set_index("date")
       ["close"])

# Gap between VIX and realized volatility
vix_gap = (vix - RV).dropna()

In [34]:
current_iv = 18.9
current_vix = 22.40

In [35]:
print(f"Date: {RV.index.union(vix.index)[-1]}")
print("-" * 50)
print(f"Mean VIX-RV gap (past 2 years): {round(vix_gap.iloc[-504:].mean(), 2)}%")
print(f"Current historical gap (21 days avg): {round(current_vix - RV.iloc[-21:].mean(), 2)}%")
print("-" * 50)
print(f"Current IV vs 2 year RV: {round(percentileofscore(RV.iloc[-504:], current_iv), 2)}%")
print("-" * 50)
print(f"IV expected daily move (1 std): {round(current_iv / np.sqrt(252), 2)}%")
print(f"IV expected daily move (2 std): {round(current_iv / np.sqrt(252) * 2, 2)}%")
print(f"IV expected daily move (3 std): {round(current_iv / np.sqrt(252) * 3, 2)}%")

Date: 2022-01-18
--------------------------------------------------
Mean VIX-RV gap (past 2 years): 11.82%
Current historical gap (21 days avg): 11.55%
--------------------------------------------------
Current IV vs 2 year RV: 85.91%
--------------------------------------------------
IV expected daily move (1 std): 1.19%
IV expected daily move (2 std): 2.38%
IV expected daily move (3 std): 3.57%
